In [1]:
import pandas as pd

In [63]:
df = pd.read_csv("../data/tws-clean.csv")

In [64]:
df[["comment", "clean_comment"]]

,comment,clean_comment
0,bagus,bagus
1,1tidak menyala,tidak nyala
2,puas brangnya sesusai,puas barang sesuai
3,lumayan lah walaupun bass nya kurang,lumayan walaupun bass kurang
4,"barang udah sampai, packing bagus, barang mulu...",barang udah packing bagus barang mulus lagi ca...
...,...,...
2312,berfungsi ke 2 nya. suara sesuai harga lumayanlah,fungsi nya suara sesuai harga lumayan
2313,mantap,mantap
2314,bagus,bagus
2315,baik,baik


In [69]:
index = df.sample(1).index

before = df.loc[index]["comment"].values[0]
after = df.loc[index]["clean_comment"].values[0]

print(f"Sebelum preprocessing:\n{before}\n{'=' * 50}")
print(f"Setelah preprocessing:\n{after}\n{'=' * 50}")


Sebelum preprocessing:
sudah dicoba. bagus suaranya. top. hanya saja ukuran nya agak gede utk telinga ku
Setelah preprocessing:
coba bagus suara baik saja ukur agak besar telinga ku
